Câu 1: Phân phối Bernoulli và Multinomial

Cho tập dữ liệu Education.csv [https://drive.google.com/file/d/1Gn6YWHXRuPbTUXY5HFxM5C_tJHuZxCka/view?usp=sharing]
- Trong đó:
    - Text: Chứa đoạn văn bản liên quan đến chủ đề giáo dục.
    - Label: Chứa nhãn cảm xúc của văn bản [Tích cực (Positive)/Tiêu cực (Negative)].
- Yêu cầu: Áp dụng thuật toán Naive Bayes (phân phối bernoulli và phân phối Multinomial) để dự đoán cảm xúc của văn bản là tích cực hay tiêu cực và so sánh kết quả của hai phân phối đó.

In [ ]:
from flask import Flask, Response
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

import numpy as np
import io
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib
matplotlib.use("SVG")
app = Flask(__name__)

@app.route('/print-plot')
def plot_png():
    file_path = '/Users/phamquangminh/Documents/tài liệu/university/HK1-Y3/Học máy và ứng dụng/Code/Education.csv'
    data = pd.read_csv(file_path)

    X = data['Text']
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(data['Label'])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    vectorizer = CountVectorizer(stop_words='english')
    X_train_bow = vectorizer.fit_transform(X_train)
    X_test_bow = vectorizer.transform(X_test)

    bernoulli_nb = BernoulliNB()
    multinomial_nb = MultinomialNB()

    bernoulli_nb.fit(X_train_bow, y_train)
    multinomial_nb.fit(X_train_bow, y_train)

    y_pred_bernoulli = bernoulli_nb.predict(X_test_bow)
    y_pred_multinomial = multinomial_nb.predict(X_test_bow)

    report_bernoulli = classification_report(y_test, y_pred_bernoulli, output_dict=True)
    report_multinomial = classification_report(y_test, y_pred_multinomial, output_dict=True)

    fig, axes = plt.subplots(1, 2, figsize=(18, 12))

    labels = list(report_bernoulli.keys())[:-3]
    metrics = ['precision', 'recall', 'f1-score', 'support']

    data_bernoulli = np.array([[report_bernoulli[label][metric] for metric in metrics] for label in labels])
    cax_bernoulli = axes[0].matshow(data_bernoulli, cmap='coolwarm')
    fig.colorbar(cax_bernoulli, ax=axes[0])
    axes[0].set_xticks(range(len(metrics)))
    axes[0].set_xticklabels(metrics)
    axes[0].set_yticks(range(len(labels)))
    axes[0].set_yticklabels(labels)
    axes[0].set_title('Bernoulli Naive Bayes')

    for (i, j), val in np.ndenumerate(data_bernoulli):
        axes[0].text(j, i, f'{val:.2f}', ha='center', va='center', color='white')

    data_multinomial = np.array([[report_multinomial[label][metric] for metric in metrics] for label in labels])
    cax_multinomial = axes[1].matshow(data_multinomial, cmap='coolwarm')
    fig.colorbar(cax_multinomial, ax=axes[1])
    axes[1].set_xticks(range(len(metrics)))
    axes[1].set_xticklabels(metrics)
    axes[1].set_yticks(range(len(labels)))
    axes[1].set_yticklabels(labels)
    axes[1].set_title('Multinomial Naive Bayes')

    for (i, j), val in np.ndenumerate(data_multinomial):
        axes[1].text(j, i, f'{val:.2f}', ha='center', va='center', color='white')

    plt.tight_layout()

    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    return Response(output.getvalue(), mimetype='image/png')

if __name__ == '__main__':
    app.run(debug=True, port=5002)


Câu 2: Phân phối Gaussian

Cho tập dữ liệu Drug.csv [https://drive.google.com/file/d/1_G8oXkLlsauQkujZzJZJwibAWu5PgBXK/view?usp=sharing]
- Trong đó:
  - Age: Tuổi của bệnh nhân
  - Sex: Giới tính của bệnh nhân
  - BP: Mức huyết áp
  - Cholesterol: Mức cholesterol trong máu
  - Na_to_K: Tỷ lệ Natri và Kali trong máu
  - Drug: Loại thuốc [A/B/C/X/Y]
- Yêu cầu: Áp dụng thuật toán Naive Bayes (phân phối Gaussian) để dự đoán kết quả loại thuốc phù hợp với bệnh nhân.

In [8]:
from flask import Flask, Response
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np
import io
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib
matplotlib.use("SVG")

app = Flask(__name__)

@app.route('/print-plot')
def plot_png():
    data = pd.read_csv('/Users/phamquangminh/Documents/tài liệu/university/HK1-Y3/Học máy và ứng dụng/Code/drug.csv')

    label_encoder_sex = LabelEncoder()
    data['Sex'] = label_encoder_sex.fit_transform(data['Sex'])

    label_encoder_BP = LabelEncoder()
    data['BP'] = label_encoder_BP.fit_transform(data['BP'])

    label_encoder_chol = LabelEncoder()
    data['Cholesterol'] = label_encoder_chol.fit_transform(data['Cholesterol'])

    X = data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']]

    y = data['Drug']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = GaussianNB()

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred,output_dict=True)


    fig, axes = plt.subplots(1, 1, figsize=(18, 12))  


    labels = list(report.keys())[:-3]  
    metrics = ['precision', 'recall', 'f1-score', 'support']


    data_bernoulli = np.array([[report[label][metric] for metric in metrics] for label in labels])
    cax_bernoulli = axes.matshow(data_bernoulli, cmap='coolwarm')
    fig.colorbar(cax_bernoulli, ax=axes)
    axes.set_xticks(range(len(metrics)))
    axes.set_xticklabels(metrics)
    axes.set_yticks(range(len(labels)))
    axes.set_yticklabels(labels)
    axes.set_title('Bernoulli Naive Bayes')


    for (i, j), val in np.ndenumerate(data_bernoulli):
        axes.text(j, i, f'{val:.2f}', ha='center', va='center', color='white')

    plt.tight_layout()


    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    return Response(output.getvalue(), mimetype='image/png')

if __name__ == '__main__':
    app.run(debug=True, port=5003)


Accuracy: 92.50%
              precision    recall  f1-score   support

       DrugY       1.00      0.80      0.89        15
       drugA       0.86      1.00      0.92         6
       drugB       0.75      1.00      0.86         3
       drugC       0.83      1.00      0.91         5
       drugX       1.00      1.00      1.00        11

    accuracy                           0.93        40
   macro avg       0.89      0.96      0.92        40
weighted avg       0.94      0.93      0.92        40

